In [1]:
import spiceypy
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Load kernels using the kernel_meta.txt file
spiceypy.furnsh("kernel_meta.txt")

Set the start and end dates

In [2]:
# Datetime object for the start and end times
init_time_utc = datetime.datetime(year=2021, month=10, day=1)
end_time_utc = datetime.datetime(year=2022, month=6, day=1)

# Convert datetime objects to strings
init_time_utc_str = init_time_utc.strftime("%Y-%m-%dT%H:%M:%S")
end_time_utc_str = end_time_utc.strftime("%Y-%m-%dT%H:%M:%S")

# Convert UTC datetime to ephemeris time
init_time_et = spiceypy.utc2et(init_time_utc_str)
end_time_et = spiceypy.utc2et(end_time_utc_str)

# Number of seconds in an hour
delta_hour_in_seconds = 3600

# Create an array between the two dates with a step every hour
time_interval_et = np.linspace(
    start=init_time_et, stop=end_time_et, num=delta_hour_in_seconds
)

Create a pandas dataframe with the times for calculation

In [3]:
# Create an empty pandas dataframe
inner_solsys_df = pd.DataFrame()

# Set the column ET that stores the ETs
inner_solsys_df.loc[:, "ET"] = time_interval_et

# Crate UTC column to set ETs back to UTC
inner_solsys_df.loc[:, "UTC"] = inner_solsys_df["ET"].apply(
    lambda x: spiceypy.et2datetime(et=x)
)

In [4]:
# NAIF Guide
## Earth : 399
## Sun : 10
## Venus : 299
## Moon : 301

inner_solsys_df.loc[:, "EARTH_VEN2SUN_ANGLE"] = inner_solsys_df["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="10", obsrvr="299", abcorr="LT+S")
    )
)

inner_solsys_df.loc[:, "EARTH_MOON2SUN_ANGLE"] = inner_solsys_df["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="10", obsrvr="301", abcorr="LT+S")
    )
)

inner_solsys_df.loc[:, "EARTH_MOON2VEN_ANGLE"] = inner_solsys_df["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="299", obsrvr="301", abcorr="LT+S")
    )
)